In [1]:
import numpy as np
from math import pi, sqrt, tanh
from matplotlib import pyplot
from scipy.linalg import block_diag
from scipy.optimize import minimize
import time
from ipywidgets import interact

In [ ]:
hbar=1.05e-34
elec=1.6e-19
meV=elec*1e-3

vF=1e6
B=10 # Tesla
hwc=vF*sqrt(2*hbar*elec*B)
lB=np.sqrt(hbar/(elec*B))
m_sc=3*hwc #1e2*meV
mu_sc=8*hwc #1e2*meV
D_sc=0.6*hwc #10*meV
nu=0.4#sqrt(2)+0.1
mu_n=nu*hwc #10*meV

lambd=.0*vF
gs=0.*hwc
gn=0.2*hwc
lRx= 1*hwc
lRy= 0*hwc
lso= 0*hwc


ky_sw=np.linspace(-1,1,80)*8/lB


Ls=4*lB
Lx=12*lB
Nx=200
kx=2*pi*np.arange(Nx)/(2*Lx+Ls)-pi*Nx/(2*Lx+Ls)
[k1,k2]=2*pi*np.mgrid[range(Nx),range(Nx)]/(2*Lx+Ls)-pi*Nx/(2*Lx+Ls)


def y2(k):
    with np.errstate(divide='ignore', invalid='ignore'):
        c= np.exp(1j*k*Ls/2)*(-2j+np.exp(1j*k*Lx)*(2j+ 2*k*Lx- 1j* k**2 * Lx**2))/ (k**3) /(2*Lx+Ls)
        c[ ~ np.isfinite( c )] = Lx**3 /(3*(2*Lx+Ls))
    return c

def y1(k):
    with np.errstate(divide='ignore', invalid='ignore'):
        c = np.exp(1j*k*Ls/2)*(-1+np.exp(1j*k*Lx)*(1- 1j* k * Lx))/ (k**2) /(2*Lx+Ls)
        c[ ~ np.isfinite( c )] = Lx**2 /(2*(2*Lx+Ls))
    return c

def y0(k):
    with np.errstate(divide='ignore', invalid='ignore'):
        c = -1j*np.exp(1j*k*Ls/2)*(-1+np.exp(1j*k*Lx) )/(k*(2*Lx+Ls))
        c[ ~ np.isfinite( c )] = Lx /(2*Lx+Ls)
    return c

def y0s(k):
    with np.errstate(divide='ignore', invalid='ignore'):
        c = 2*np.sin(k*Ls/2)/(k*(2*Lx+Ls))
        c[ ~ np.isfinite( c )] = Ls /(2*Lx+Ls)
    return c

sigma0=np.array([[1,0],[0,1]])
sigma1=np.array([[0,1],[1,0]])
sigma2=np.array([[0,-1j],[1j,0]])
sigma3=np.array([[1,0],[0,-1]])

Hx= hbar*vF*np.kron(np.diag(kx) , sigma1)
HeB= 2j*vF*elec*B* np.kron(np.imag(y1(k1-k2)), sigma2)
HeBT= 2j*vF*elec*B* np.kron(np.imag(y1(k2-k1)), sigma2)

Hxm= np.kron( y0s(k1-k2) , m_sc*sigma3)
HxmT= np.kron( y0s(k2-k1) , m_sc*sigma3)
Hef= -np.kron( 2*mu_n*np.real(y0(k1-k2)) + mu_sc*y0s(k1-k2), sigma0 )
HefT= -np.kron( 2*mu_n*np.real(y0(k2-k1)) + mu_sc*y0s(k2-k1), sigma0 )

Hd= np.kron(D_sc*y0s(k1-k2),sigma0)
# basis ud,ud
dmat=np.array([[0.,1.],[0.,0.]])
ddmat=np.kron(dmat,np.array([[0.,1.],[-1.,0.]]))
Hd_mat=np.kron(ddmat,Hd)
Hd_mat=Hd_mat+np.transpose(Hd_mat).conj()

# Hlx=hbar*lambd*np.kron(np.array([[0.,-1j],[1j,0.]]), np.kron(k1*y0s(k1-k2),sigma0) )
# HlxT=-hbar*lambd*np.kron(np.array([[0.,-1j],[1j,0.]]), np.kron(k1*y0s(k2-k1),sigma0) )
Hlx= np.kron(sigma2,np.kron( y0s(k1-k2) , lRx*sigma1)) #1
HlxT= np.kron(sigma2,np.kron( y0s(k2-k1) , lRx*sigma1)) #1
Hly= -np.kron(sigma1,np.kron( y0s(k1-k2) , lRy*sigma2)) #2
HlyT= -np.kron(sigma1,np.kron( y0s(k2-k1) , lRy*sigma2)) #2
Hlz= np.kron(sigma3,np.kron( y0s(k1-k2) , lso*sigma3)) # 333
HlzT= np.kron(sigma3,np.kron( y0s(k2-k1) , lso*sigma3))
Hl=Hlx+Hly+Hlz
HlT=HlxT+HlyT+HlzT
Hz_up= -(gs*y0s(k1-k2)+2*gn*np.real(y0(k1-k2)))
Hz= np.kron(sigma3, np.kron(Hz_up,sigma0) )
Hz_upT= -(gs*y0s(k2-k1)+2*gn*np.real(y0(k2-k1)))
HzT= np.kron(sigma3, np.kron(Hz_upT,sigma0) )

Vn=np.zeros((8*Nx,8*Nx,len(ky_sw)), dtype=np.complex64)
En=np.zeros((8*Nx,len(ky_sw)))

for i_k in range(len(ky_sw)):
    ky=ky_sw[i_k]
    Hy= np.kron(hbar*vF*ky*np.eye(Nx), sigma2)

    H1 = Hx +Hy + HeB + Hxm + Hef
    H1T = -Hx +Hy + (HeBT + HxmT + HefT).conj()

#         Hly=hbar*lambd*ky*np.kron(np.array([[0.,1],[1,0.]]), np.kron(y0s(k1-k2),sigma0) )
#         HlyT=-hbar*lambd*ky*np.kron(np.array([[0.,1],[1,0.]]), np.kron(y0s(k2-k1),sigma0) )

    H_p=block_diag(H1,H1)+Hl+Hz
    H_pT=block_diag(H1T,H1T)+(HlT+HzT).conj()

    H_t=block_diag(H_p,-H_pT)+Hd_mat
    En[:,i_k], Vn[:,:,i_k] =np.linalg.eigh(H_t)

print("Done!")

In [ ]:
pyplot.figure()
pyplot.plot(ky_sw*lB,En.T/hwc,'b')
pyplot.ylabel(r"En [$\Delta$]")
pyplot.xlabel(r"$X[l_B]$")
# pyplot.ylim(-22,22)
# pyplot.ylim(-0.015,0.015)
pyplot.ylim(-1,1)
pyplot.grid()

In [ ]:
def wf_real_space(ev,k,x,Ls):
    Nx=len(k)
    kt=np.reshape(k,[1,Nx])
    xt=np.reshape(x,[len(x),1])
    
    ev_p= ev[:4*Nx,:]
    ev_h= ev[4*Nx:,:]
    ## uA # uB # dA # dB
    return np.dot(np.exp(1j*kt*xt),ev_p[0:2*Nx:2,:])/np.sqrt(2*Lx+Ls)*np.sqrt(x[1]-x[0]),\
           np.dot(np.exp(1j*kt*xt),ev_p[1:2*Nx:2,:])/np.sqrt(2*Lx+Ls)*np.sqrt(x[1]-x[0]),\
           np.dot(np.exp(1j*kt*xt),ev_p[2*Nx:4*Nx:2,:])/np.sqrt(2*Lx+Ls)*np.sqrt(x[1]-x[0]),\
           np.dot(np.exp(1j*kt*xt),ev_p[2*Nx+1:4*Nx:2,:])/np.sqrt(2*Lx+Ls)*np.sqrt(x[1]-x[0]),\
           np.dot(np.exp(1j*kt*xt),ev_h[0:2*Nx:2,:])/np.sqrt(2*Lx+Ls)*np.sqrt(x[1]-x[0]),\
           np.dot(np.exp(1j*kt*xt),ev_h[1:2*Nx:2,:])/np.sqrt(2*Lx+Ls)*np.sqrt(x[1]-x[0]),\
           np.dot(np.exp(1j*kt*xt),ev_h[2*Nx:4*Nx:2,:])/np.sqrt(2*Lx+Ls)*np.sqrt(x[1]-x[0]),\
           np.dot(np.exp(1j*kt*xt),ev_h[2*Nx+1:4*Nx:2,:])/np.sqrt(2*Lx+Ls)*np.sqrt(x[1]-x[0])


def plot_wf(i_band=Nx,iwf=int(len(ky_sw)/2)):
# iwf=0
    pyplot.figure(figsize=(10,5))
    pyplot.subplot(2,2,1)
    pyplot.plot(ky_sw*lB,En.T/hwc,'b')
    pyplot.plot(ky_sw[iwf]*lB,En[i_band,iwf]/hwc,'ro')
    pyplot.ylabel("Energy(meV)")
    pyplot.xlabel("X(l)")
    pyplot.ylim(-0.1,0.1)
    # pyplot.xlim(-7.5,7.5)
    pyplot.grid()
#     pyplot.show()

    x0=np.linspace(-1,1,500)*10*lB
    wfp_uA,wfp_uB, wfp_dA,wfp_dB,\
    wfh_uA,wfh_uB, wfh_dA,wfh_dB =wf_real_space(np.reshape(Vn[:,i_band,:],[8*Nx,len(ky_sw)]),kx,x0,Ls)
#     pyplot.figure()
    pyplot.title('ky=%.2f' % (ky_sw[iwf]*lB) )
    pyplot.subplot(2,2,3)
    pyplot.plot(x0/lB,np.abs(wfp_uA[:,iwf])**2,label=r'$eA\uparrow$')
    pyplot.plot(x0/lB,np.abs(wfp_uB[:,iwf])**2,label=r'$eB\uparrow$')
    pyplot.plot(x0/lB,np.abs(wfh_dA[:,iwf])**2,label=r'$hA\downarrow$')
    pyplot.plot(x0/lB,np.abs(wfh_dB[:,iwf])**2,label=r'$hB\downarrow$')
    pyplot.legend()
    pyplot.subplot(2,2,4)
    pyplot.plot(x0/lB,np.abs(wfp_dA[:,iwf])**2,label=r'$eA\downarrow$')
    pyplot.plot(x0/lB,np.abs(wfp_dB[:,iwf])**2,label=r'$eB\downarrow$')
    pyplot.plot(x0/lB,np.abs(wfh_uA[:,iwf])**2,label=r'$hA\uparrow$')
    pyplot.plot(x0/lB,np.abs(wfh_uB[:,iwf])**2,label=r'$hB\uparrow$')
    pyplot.legend()
    pyplot.show()

interact(plot_wf,iwf=(0,len(ky_sw)-1,1), i_band=range(4*Nx-int(nu**2)-3,4*Nx+int(nu**2)+3));
#, continuous_update=False);